**Homework 2:**
Top university rankings


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
topuni_url = 'https://www.topuniversities.com/university-rankings/world-university-rankings/2018'
r = requests.get(topuni_url)
soup = BeautifulSoup(r.text,'html.parser')
soup.title

<title>QS World University Rankings 2018 | Top Universities</title>

In [20]:
text_url = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508404716580'
r = requests.get(text_url)
soup = BeautifulSoup(r.text,'html.parser')
q = r.json()
q['data'][0]
type(q)

dict